# Author: Justin Hsi
# With usage of dtyping, able to combine all the pmt_history parts into this one notebook

# Part  1 of cleaning lending club payment history

In [1]:
import dir_constants as dc
from pandas.api.types import CategoricalDtype
from tqdm import tqdm_notebook
import data_science.lendingclub.dataprep_and_modeling.modeling_utils.data_prep_new as data_prep

store_path = '/home/justin/lendingclub.h5'

data_path = '/home/justin/rsync_dl_rig/unzipped_lc_csvs/'
files = os.listdir(data_path)
filename = [file_ for file_ in files if file_.startswith('PMTHIST')]
if len(filename) > 1:
    print('you need to delete old pmt history files')
else:
    filename = filename[0]
# con = sqlite3.connect("/home/justin/lendingclub.db3")
# chunksize = 1e6

date_cols = ['RECEIVED_D', 'IssuedDate', 'MONTH', 'EarliestCREDITLine']

month_dict = {
    'JAN': '1-',
    'FEB': '2-',
    'MAR': '3-',
    'APR': '4-',
    'MAY': '5-',
    'JUN': '6-',
    'JUL': '7-',
    'AUG': '8-',
    'SEP': '9-',
    'OCT': '10-',
    'NOV': '11-',
    'DEC': '12-'
}


def check_mem():
    # These are the usual ipython objects, including this one you are creating
    ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

    # Get a sorted list of the objects and their sizes
    print(sorted([(x, sys.getsizeof(globals().get(x))) for x in globals() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True))

In [2]:
check = pd.read_csv(f'{data_path}{filename}', nrows=2500000)
obj_cols = check.select_dtypes(include=['object']).describe()

/home/justin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# set up category dtype where possible, keep track of cols with nans/stupid
# cols: Inquiries6m has strings, nans, and ints. Dumb.
stupid_cols = []
dtype = {}
for col in obj_cols.columns:
    if col not in date_cols:
        try:
            vals = check[col].unique()
            vals.sort()
            dtype[col] = CategoricalDtype(vals)
        except:
            stupid_cols.append(col)
            print(col)
            print(check[col].unique())

Inquiries6M
['0' '1' '4' '5' '*' '8' '2' '3' nan '9' '6' '7' 0 1 3 4 2 5 8 6 9 7]
EmploymentLength
['< 1 year' '10+ years' '1 year' '6 years' '9 years' '3 years' '5 years'
 '7 years' '2 years' '8 years' '4 years' nan]


In [4]:
# manual fixes to existing categoricals:
for k,v in dtype.items():
    if k == 'APPL_FICO_BAND':
        dtype[k] = CategoricalDtype(v.categories, ordered=True)
    elif k == 'HomeOwnership':
        pass
    elif k == 'Last_FICO_BAND':
        if list(v.categories)[-1] != '845-HIGH':
            miss_low = list(v.categories[-2:][::-1])
            rest = list(v.categories[:-2])
            dtype[k] = CategoricalDtype(miss_low+rest, ordered=True)
    elif k == 'PERIOD_END_LSTAT':
        pass
    elif k == 'State':
        pass
    elif k == 'VINTAGE':
        dtype[k] = CategoricalDtype(v.categories, ordered=True)
    elif k == 'grade':
        dtype[k] = CategoricalDtype(v.categories, ordered=True)        

In [5]:
for col in check.columns:
    if col not in dtype.keys():
        if col not in date_cols and col != 'LOAN_ID' and col not in stupid_cols:
            dtype[col] = np.float32
del check

In [6]:
pmt_hist=pd.read_csv(f'{data_path}{filename}', dtype=dtype, na_values='*') #dtype=dtype, 

# changing dtypes and then renaming

In [7]:
# fixing dates to be sensible dtypes
for col in date_cols:
    pmt_hist[col] = pd.to_datetime(
    pmt_hist[col].str[:3].replace(month_dict) +
    pmt_hist[col].str[3:],
    format='%m-%Y')
    
# changing to float32    
pmt_hist['Inquiries6M'] = pmt_hist['Inquiries6M'].astype(np.float32)    

# fixing up Employment Length to categorical
# pmt_hist['EmploymentLength'] = pmt_hist['EmploymentLength'].fillna('MISSING')
emp_len_dtype = CategoricalDtype(['< 1 year', '1 year', '2 years',
                                  '3 years', '4 years', '5 years', '6 years',
                                  '7 years', '8 years', '9 years', '10+ years'], ordered=True)
pmt_hist['EmploymentLength'] = pmt_hist['EmploymentLength'].astype(emp_len_dtype)

In [9]:
# renaming columns ____________________________________________________________
rename_col_dict = {
    'LOAN_ID': 'loan_id',
    'PBAL_BEG_PERIOD': 'outs_princp_beg',
    'PRNCP_PAID': 'princp_paid',
    'INT_PAID': 'int_paid',
    'FEE_PAID': 'fee_paid',
    'DUE_AMT': 'amt_due',
    'RECEIVED_AMT': 'amt_paid',
    'RECEIVED_D': 'pmt_date',
    'PERIOD_END_LSTAT': 'status_period_end',
    'MONTH': 'date',
    'PBAL_END_PERIOD': 'outs_princp_end',
    'MOB': 'm_on_books',
    'CO': 'charged_off_this_month',
    'COAMT': 'charged_off_amt',
    'InterestRate': 'int_rate',
    'IssuedDate': 'issue_date',
    'MONTHLYCONTRACTAMT': 'monthly_pmt',
    'dti': 'dti',
    'State': 'addr_state',
    'HomeOwnership': 'home_ownership',
    'MonthlyIncome': 'm_income',
    'EarliestCREDITLine': 'first_credit_line_date',
    'OpenCREDITLines': 'open_credit_lines',
    'TotalCREDITLines': 'total_credit_lines',
    'RevolvingCREDITBalance': 'revol_credit_bal',
    'RevolvingLineUtilization': 'revol_line_util',
    'Inquiries6M': 'inq_6m',
    'DQ2yrs': 'dq_24m',
    'MonthsSinceDQ': 'm_since_dq',
    'PublicRec': 'public_recs',
    'MonthsSinceLastRec': 'm_since_rec',
    'EmploymentLength': 'emp_len',
    'currentpolicy': 'current_policy',
    'grade': 'grade',
    'term': 'term',
    'APPL_FICO_BAND': 'fico_apply',
    'Last_FICO_BAND': 'fico_last',
    'VINTAGE': 'vintage',
    'PCO_RECOVERY': 'recovs',
    'PCO_COLLECTION_FEE': 'recov_fees',
}

pmt_hist.rename(columns=rename_col_dict, inplace=True)

# look at columns with null in them, figure out what I want to do with them

In [10]:
def find_null_cols(df):
    cols_with_nulls = []
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            cols_with_nulls.append(col)
    return cols_with_nulls

In [11]:
cols_with_nulls = find_null_cols(pmt_hist)

In [12]:
def make_isnull_col(cols_with_nulls, df):
    '''cols_with_nulls is list of dataframe column names with nulls'''
    for col in cols_with_nulls:
        df[col+'_isnull'] = np.where(df[col].isnull(),1,0).astype(np.float32)
    print('done making isnull columns for cols with nulls')

In [13]:
make_isnull_col(cols_with_nulls, pmt_hist)

done making isnull columns for cols with nulls


In [14]:
# Check how many unique loan ids
print('payment history for', len(pmt_hist['loan_id'].unique()), 'different loan ids')

payment history for 1646778 different loan ids


In [15]:
pmt_hist.head()

,loan_id,outs_princp_beg,princp_paid,int_paid,fee_paid,amt_due,amt_paid,pmt_date,status_period_end,date,...,revol_line_util_isnull,inq_6m_isnull,dq_24m_isnull,m_since_dq_isnull,public_recs_isnull,m_since_rec_isnull,emp_len_isnull,vintage_isnull,recovs_isnull,recov_fees_isnull
0,54734,25000.000000,581.297913,247.802078,0.0,829.099976,829.099976,2009-09-01,Current,2009-09-01,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
1,54734,24418.701172,587.059814,242.040207,0.0,829.099976,829.099976,2009-10-01,Current,2009-10-01,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
2,54734,23831.642578,592.878784,236.221222,0.0,829.099976,829.099976,2009-11-01,Current,2009-11-01,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
3,54734,23238.763672,598.755432,230.344559,0.0,829.099976,829.099976,2009-12-01,Current,2009-12-01,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
4,54734,22640.007812,604.690369,224.409653,0.0,829.099976,829.099976,2010-01-01,Current,2010-01-01,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0


In [16]:
pmt_hist.to_hdf(store_path, key='pmt_hist_temp1', format='table')

In [17]:
store = pd.HDFStore(store_path)
pmt_hist = store['pmt_hist_temp1']
store.close()

# Making sure columns make logical sense

In [18]:
# There are 5 columns dealing with money: princp_paid, int_paid, fee_paid,
# recovs, and recovs_fee. princp_paid + int_paid + fee_paid is sometimes short
# of amt_paid. Be conservative and rewrite amt_paid to be sum of said 3.
# Also make all_cash_to_inv = amt_paid + recovs - recov_fees
# Fee paid is always positive, and by inspection it is money borrower pays out
pmt_hist[
    'amt_paid'] = pmt_hist['princp_paid'] + pmt_hist['int_paid'] + pmt_hist['fee_paid']
pmt_hist['recovs'].fillna(0, inplace=True)
pmt_hist['recov_fees'].fillna(0, inplace=True)
pmt_hist[
    'all_cash_to_inv'] = pmt_hist['amt_paid'] + pmt_hist['recovs'] - pmt_hist['recov_fees']

In [19]:
# status_period_end ____________________________________________________________
status_fix = {
    'Current': 'current',
    'Late (31-120 days)': 'late_120',
    'Fully Paid': 'paid',
    'Charged Off': 'charged_off',
    'Default': 'defaulted',
    'Late (16-30 days)': 'late_30',
    'In Grace Period': 'grace_15',
    'Issued': 'issued'
}
pmt_hist['status_period_end'].replace(status_fix, inplace=True)
status_fix_cats = CategoricalDtype(list(pmt_hist['status_period_end'].unique()))
pmt_hist['status_period_end'] = pmt_hist['status_period_end'].astype(status_fix_cats)

In [20]:
# fix on a few bad rows where I think there is a mistaken amt_paid ____________
pmt_hist.ix[(pmt_hist['pmt_date'].isnull() & pmt_hist['amt_paid'] > 0),
            'amt_paid'] = 0

/home/justin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


# Store part 1 of cleaning

In [21]:
pmt_hist.to_hdf(store_path, key='pmt_hist_temp1', format='table')

# Part 2, removing duplicate date rows (compressing granulairty of data to monthly, as that's the only real way I can work with it currently)

In [2]:
pmt_hist = pd.read_hdf(store_path, key='pmt_hist_temp1')

In [3]:
def test_pd_NaT(x):
    return type(x) == type(pd.tslib.NaT)

v_test_pd_NaT = np.vectorize(test_pd_NaT)

def find_dupe_dates(group):
    return pd.to_datetime(group[group.duplicated('date')]['date'].values)

def merge_dupe_dates(group, column_iloc_map, dtypes_map):
    df_chunks = []
    dupe_dates = find_dupe_dates(group)
    # add the good rows
    df_chunks.append(group[~group['date'].isin(dupe_dates)])
    
    #sum up duplicate rows
    for date in dupe_dates:
        problem_rows = group[group['date'] == date]
        ori_index = problem_rows.index
        keep_row = problem_rows.iloc[-1].to_dict()
        keep_row['outs_princp_beg'] = problem_rows.ix[ori_index[0],column_iloc_map['outs_princp_beg']]
        
        summed = problem_rows.sum()
        keep_row['princp_paid'] = summed['princp_paid']
        keep_row['int_paid'] = summed['int_paid']
        keep_row['fee_paid'] = summed['fee_paid']
        keep_row['amt_due'] = summed['amt_due']
        keep_row['amt_paid'] = summed['amt_paid']
        keep_row['charged_off_amt'] = summed['charged_off_amt']
        keep_row['recovs'] = summed['recovs']
        keep_row['recov_fees'] = summed['recov_fees']
            

        df_chunks.append(pd.DataFrame(pd.Series(keep_row),columns=[ori_index[-1]]).T)
    fixed = pd.concat(df_chunks)
    
#     import pdb; pdb.set_trace()
    
    try:
        return fixed.astype(dtypes_map)
    except:
        for col in fixed.columns:
#             series = fixed[col].copy()
# #             series[np.apply_along_axis(v_test_pd_NaT, 0, series)] = np.nan
#             series = pd.to_datetime(series)
            try:
                fixed[col] = fixed[col].astype(dtypes_map[col])
            except:
                fixed[col] = pd.to_datetime(fixed[col])
        return fixed

In [4]:
print('{0} groups to iterate through (for each loan id)'.format(len(pmt_hist['loan_id'].unique())))

1646778 groups to iterate through (for each loan id)


In [5]:
loans_with_two_entries_in_same_month = pmt_hist[pmt_hist.duplicated(
    ['loan_id', 'date'])]
dup_date_ids = loans_with_two_entries_in_same_month['loan_id'].unique()

In [6]:
already_good = pmt_hist[~pmt_hist['loan_id'].isin(dup_date_ids)]
needs_fixing = pmt_hist[pmt_hist['loan_id'].isin(dup_date_ids)]

id_grouped = needs_fixing.groupby('loan_id')

In [7]:
column_iloc_map = {
            col_name: pmt_hist.iloc[-1].index.get_loc(col_name)
            for col_name in pmt_hist.columns.values
        }
dtypes_map = already_good.dtypes.to_dict()

In [8]:
fixed_dfs = []
for ids, group in tqdm_notebook(id_grouped):
    if ids in dup_date_ids:
        fixed_dfs.append(merge_dupe_dates(group, column_iloc_map, dtypes_map))
    else:
        pass

A Jupyter Widget

/home/justin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [10]:
all_dfs = [already_good]
all_dfs.extend(fixed_dfs)

In [11]:
pmt_hist = pd.concat(all_dfs)

In [16]:
# store
pmt_hist.to_hdf(store_path, key='pmt_hist_temp2', format='table')

# Part 3, Adding a row where money related values are 0 for some loans that are missing records in a month (for npv calculation later

In [ ]:
pmt_hist = pd.read_hdf(store_path, key='pmt_hist_temp2')

In [17]:
def find_closest_previous_record(ids, issue_d, first_date, actual_months, month):
    '''This function finds the closest previous month that is in the group. 
    It is here to handle cases where a record of one month is missing, but the
    record before that missing month is also missing.'''
    offset = pd.DateOffset(months=-1)
    prev_month = month + offset
    if month < issue_d:
        print(ids)
        return first_date
    elif prev_month in actual_months:
        return prev_month
    else:
        find_closest_previous_record(ids, issue_d, first_date, actual_months, prev_month)

In [22]:
fixed_dfs = []
fixed_ids = []
id_grouped = pmt_hist.groupby('loan_id')
for ids, group in tqdm_notebook(id_grouped):
    # Copy Paste finished below
    issue_d = group['issue_date'].min()
    first_date = group['date'].min()
    last_date = group['date'].max()
    expected_months = set(pd.DatetimeIndex(start=first_date, end=last_date, freq='MS'))
    actual_months = set(group['date'])
    to_make_months = list(expected_months.symmetric_difference(actual_months))
    to_make_months.sort()
    if len(to_make_months) > 1:
        fixed_ids.append(ids)
        months_to_copy = []
        for month in to_make_months:
            months_to_copy.append(find_closest_previous_record(ids, issue_d, first_date, actual_months, month))
        copied = group[group['date'].isin(months_to_copy)].copy()
        copied['amt_paid'] = 0.0
        copied['date'] = to_make_months
        copied['amt_due'] = np.where(copied['date'] < first_date, 0, copied['amt_due'])
        fixed_dfs.append(pd.concat([group, copied]))
    else:
        pass
#         already_good_dfs.append(group)
#         if len(already_good_dfs) == chunksize:
#             better_sized_already_good_dfs.append(pd.concat(already_good_dfs))
#             already_good_dfs = []
#         if n+1 == n_chunks: # if on the last chunk
#             better_sized_already_good_dfs.append(pd.concat(already_good_dfs))
#             already_good_dfs = []

A Jupyter Widget

In [37]:
already_good_df = pmt_hist[~pmt_hist['loan_id'].isin(fixed_ids)]

In [38]:
all_dfs = [already_good_df] + [pd.concat(fixed_dfs)]
pmt_hist = pd.concat(all_dfs)

In [47]:
pmt_hist.to_hdf(store_path, key='pmt_hist_ready', format='table')

# Below was investigating to base my choices above on

In [ ]:
# pmt_hist['unaccounted_rec_pmt_money'] = pmt_hist['amt_paid'] - (
#     pmt_hist['princp_paid'] + pmt_hist['int_paid'] + pmt_hist['fee_paid'])

# # Don't care about differences less than a cent _______________________________
# pmt_hist['unaccounted_rec_pmt_money'] = np.where(
#     pmt_hist['unaccounted_rec_pmt_money'] < 0.01, 0,
#     pmt_hist['unaccounted_rec_pmt_money'])

# # These should probably be Received_amt 0 because there is no received_d
# pmt_hist[(pmt_hist['unaccounted_rec_pmt_money'] > 0) & (pmt_hist['PERIOD_END_LSTAT'] != 'Fully Paid')]

In [ ]:


# # home_ownership _______________________________________________________________
# home_ownership_fix = {
#     'admin_us': 'other',
#     'mortgage': 'mortgage',
#     'rent': 'rent',
#     'own': 'own',
#     'other': 'other',
#     'none': 'none',
#     'any': 'none'
# }
# pmt_hist['home_ownership'] = pmt_hist['home_ownership'].str.lower().replace(
#     home_ownership_fix)



In [ ]:

# # public_recs __________________________________________________________________
# records_fix = {
#     '*': 1
# }  #leave nan as nan, but * had at least 1 from m_since_record
# pmt_hist['public_recs'] = pmt_hist['public_recs'].replace(records_fix).astype(
#     float)



In [ ]:

# # fico_apply __________________________________________________________________
# fico_apply_fix = {'850': '850-850'}
# pmt_hist['fico_apply'] = pmt_hist['fico_apply'].replace(fico_apply_fix)
# pmt_hist['fico_apply'] = (pmt_hist['fico_apply'].str[:3].astype(int) +
#                           pmt_hist['fico_apply'].str[4:].astype(int)) / 2
# pmt_hist['fico_apply'] = pmt_hist['fico_apply'].astype(int)

# # fico_last ___________________________________________________________________
# fico_last_fix = {'845-HIGH': '845-849', 'LOW-499': '495-499'}
# pmt_hist['fico_last'] = pmt_hist['fico_last'].replace(fico_last_fix)
# pmt_hist.ix[pmt_hist['fico_last'] != 'MISSING', 'fico_last'] = (
#     pmt_hist.ix[pmt_hist['fico_last'] != 'MISSING', 'fico_last'].str[:3]
#     .astype(int) + pmt_hist.ix[pmt_hist['fico_last'] != 'MISSING',
#                                'fico_last'].str[4:].astype(int)) / 2
# pmt_hist.ix[pmt_hist['fico_last'] == 'MISSING', 'fico_last'] = pmt_hist.ix[
#     pmt_hist['fico_last'] == 'MISSING', 'fico_apply']
# pmt_hist['fico_last'] = pmt_hist['fico_last'].astype(int)



In [ ]:

# # revol_credit_bal ____________________________________________________________
# pmt_hist['revol_credit_bal'] = pmt_hist['revol_credit_bal'].astype(
#     float)



In [27]:



# # There is a problem with the inquiries 6m column. Some are nan values and some
# # are marked '*' with no explanation. inq6m should be in loan info so dropping
# pmt_hist.drop('inq_6m', axis=1, inplace=True)



# Chunking example

In [29]:
# # store pmt_history in hdfstore _______________________________________________
# store = pd.HDFStore(
#     dc.home_path+'/justin_tinkering/data_science/lendingclub/{0}_store.h5'.
#     format(platform),
#     append=True)

# # Create min_itemsize_dict for allocating size when storing ___________________
# min_itemsize_dict = {}
# for col in pmt_hist.columns:
#     if pmt_hist[col].dtype == np.object:
#         print(col, pmt_hist[col].str.len().max())
#         if col in ['State', 'VINTAGE', 'grade']:
#             pass
#         else:
#             min_itemsize_dict[col] = 15

# total_len = len(pmt_hist)
# chunk_size = 120000 # 120k rows at a time
# chunks = np.ceil(total_len/chunk_size)
# df_chunks = np.array_split(pmt_hist, chunks)

     
# k = 0
# for chunk in tqdm_notebook(df_chunks):
#     if k == 0:
#         store.append(
#             'pmt_hist_intermediary_1',
#             chunk,
#             data_columns=True,
#             index=True,
#             append=False,
#             min_itemsize=min_itemsize_dict)
#         k += 1
#     else:
#         store.append(
#             'pmt_hist_intermediary_1',
#             chunk,
#             data_columns=True,
#             index=True,
#             append=True)            
        
# # store pmt_hist ids        
# pmt_hist_ids = pd.Series(pmt_hist['loan_id'].unique())
# pmt_hist_ids.to_hdf(store, 'pmt_hist_ids', mode='w')        
# print(store.keys())
# print(store)        
# store.close()